# Distortion analysis
___

In [5]:
import os
import cv2
import tifffile
import matplotlib
from numpy import ma
import numpy; np=numpy
from skimage import color, filters
from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt; plt.ion()
from skimage.exposure import rescale_intensity

%matplotlib qt5

## Helper functions
___

In [6]:
# setup colormaps for later display
cmap = numpy.zeros((256,4))
cmap[:,3] = 1
cmap[:,0] = numpy.linspace(0, 1,256)
cmap[:,2] = numpy.linspace(0, 1,256)
magenta = matplotlib.colors.ListedColormap(cmap)

cmap = numpy.zeros((256,4))
cmap[:,3] = 1
cmap[:,1] = numpy.linspace(0, 1,256)
green = matplotlib.colors.ListedColormap(cmap)

# calculation of deformation field
def calc_flow(img1, img2):
    flow = cv2.calcOpticalFlowFarneback(img1, img2, None, 0.5, 
                                        levels=3, 
                                        winsize=winsize, 
                                        iterations=3, 
                                        poly_n=5, 
                                        poly_sigma=poly_sigma, 
                                        flags=0)
    u = flow[...,0]
    v = flow[...,1]
    
    return u, v

# display subsampled deformation field with arrows
def show_flow_field(u, v, ax=None, subsample=32, color='r', scale=1/2):
    if ax is None:
        ax = plt.gca()
    x_, y_ = numpy.meshgrid(range(0, u.shape[1]), range(0, u.shape[0]))
    ax.quiver( x_[subsample//2::subsample, subsample//2::subsample], 
               y_[subsample//2::subsample, subsample//2::subsample], 
               u [subsample//2::subsample, subsample//2::subsample], 
               v [subsample//2::subsample, subsample//2::subsample],  
              angles='xy', 
              scale_units='xy', 
              scale=scale, 
              pivot='mid', 
              headwidth=2,
              headlength=3,
              color=color,
              width=0.002)

## Inputs
___

In [11]:
img_pair_in = tifffile.imread("H:\\projects\\027_sven_danzl\\alignments\\180717_01\\best_match_c0.560_ef8.100_a155.0_high-post-res_crop_magenta_green.tif")
pixel_size = 0.57


subsample   = 64
winsize     = 41
poly_sigma  = 3
post_image_sigma = 4

## Compute local distortionfield
___

In [14]:
img_pre   = img_pair_in[0]
img_post  = img_pair_in[1]

img_post = filters.gaussian(img_post, post_image_sigma, preserve_range=True)
img_post = rescale_intensity(img_post, (img_post.min(), numpy.percentile(img_post, 99.9)), 'uint8')
img_pre  = rescale_intensity(img_pre,  (img_pre.min(),  numpy.percentile(img_pre,  99.9)), 'uint8')

u, v = calc_flow(img_pre, img_post)

f = plt.figure(figsize=(10, 10))

ax = plt.Axes(f, [0,0,1,0.95],
                 yticks=[],
                 xticks=[],
                 frame_on=False)
f.add_axes(ax)
ax = plt.gca()

tmp = numpy.zeros(img_post.shape + (3,), dtype=numpy.uint8)
tmp[..., 0] = img_pre
tmp[..., 1] = img_post

ax.imshow(img_pre, cmap=green, alpha=1)
ax.imshow(img_post, cmap=magenta, alpha=0.5)

ax.set_axis_off()

U = ma.masked_array(u, mask=img_post==img_post.min())
V = ma.masked_array(v, mask=img_post==img_post.min())

show_flow_field(U, V, ax, color='w', scale=1., subsample=subsample)

## Show histogram of deformations and angular distribution
___

In [15]:
mag = numpy.sqrt(u**2 + v**2)
mag_um = mag * pixel_size

f = plt.figure(figsize=(4, 3))
hist_data = plt.hist(mag_um.ravel(), numpy.linspace(0,20,32), density=True)
plt.xlabel("Distortions (um)")
plt.ylabel("Normalized fraction (au)")
plt.xticks(range(0,21,5))
plt.tight_layout()

###
f = plt.figure(figsize=(4, 4))
bins = numpy.linspace(-numpy.pi, numpy.pi, 360)
his = plt.hist(numpy.angle(u + 1j * v).ravel(), bins, density=True)
ax = plt.subplot(111, polar=True)
ax.plot(his[1][:-1], his[0], 'r-')
plt.fill_between(his[1][:-1], 0, his[0], alpha=0.2)
plt.xlabel("Angular distribution of distortion vectors")
plt.tight_layout()
ax.set_yticklabels([])

C:\Users\csommer\anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


[]